# Ejemplo de uso de CNN
## Obtenido de: https://github.com/tensorchiefs/dl_book/blob/master/chapter_02/nb_ch02_03.ipynb
***
Para predicción de imágenes con bordes verticales


In [ ]:
# módulos necesarios
import numpy as npaqa
import matplotlib.pyplot as plt

import tensorflow.keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten , Activation
from tensorflow.keras.utils import to_categorical 

## Generación de imágenes

In [ ]:
# función que genera imágenes sintéticas con bordes
# verticales y bordes horizontales

def generate_image_with_bars(size, bar_nr, vertical = True):
    # matriz de de sizexsizex1 (1 canal)
    img=np.zeros((size,size,1),dtype="uint8")
    for i in range(0,bar_nr):
        x,y = np.random.randint(0,size,2)
        l  = int(np.random.randint(y,size,1))
    if (vertical):
        img[y:l,x,0]=255 #agrega bordes vercticales
    else:
        img[x,y:l,0]=255 #agrega bordes horizontales
    return img  

## Visualización de imágenes

In [ ]:
# visualización de dos imágenes

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
img=generate_image_with_bars(50,10, vertical=True)
plt.imshow(img[:,:,0],cmap='gray')
plt.subplot(1,2,2)
img=generate_image_with_bars(50,10, vertical=False)
plt.imshow(img[:,:,0],cmap='gray')

In [ ]:
pixel=50  # define alto y ancho de las imagenes
num_images_train = 1000 #número de datos de entrenamiento 
num_images_val = 1000 # número de imágenes de validación

# generación de datos de entrenamiento con bordes verticales
X_train =np.zeros((num_images_train,pixel,pixel,1))
for i in range(0, int(num_images_train/2)):
    X_train[i]=generate_image_with_bars(pixel,10)

# generación de datos de entrenamiento con bordes horizontales
for i in range(int(num_images_train/2), num_images_train):
    X_train[i]=generate_image_with_bars(pixel,10, vertical=False)

# generación de datos de validación con bordes verticales
X_val =np.zeros((num_images_train,pixel,pixel,1))
for i in range(0, int(num_images_train/2)): 
    X_val[i]=generate_image_with_bars(pixel,10)

# generación de datos de validación con bordes horizontales
for i in range(int(num_images_train/2), num_images_train):
    X_val[i]=generate_image_with_bars(pixel,10, vertical=False)


# Normalización de los datos entre 0 y 1
X_train=X_train/255
X_val=X_val/255

print(X_train.shape)
print(X_val.shape)

In [ ]:
# Generación de etiquetas

y = np.array([[0],[1]])
# ejemplo
# >> x = np.array([[1,2],[3,4]])
# >> np.repeat(x, 2)
# array([1, 1, 2, 2, 3, 3, 4, 4])
Y_train = np.repeat(y, num_images_train//2)
Y_val = np.repeat(y, num_images_train//2)

In [ ]:
print(Y_train.shape)
print(Y_train)

In [ ]:
# one-hot-encoding
Y_train=to_categorical(Y_train,2)
Y_val=to_categorical(Y_val,2)

In [ ]:
Y_train

## Red Convolucional 

In [ ]:
model = Sequential()
model.add(Convolution2D(1,(5,5),padding='same',input_shape=(pixel,pixel,1)))
model.add(Activation('linear'))

# toma el valor máximo del mapa de características
model.add(MaxPooling2D(pool_size=(pixel,pixel)))

# aplanamiento
model.add(Flatten()) 

# capa densa con dos neuronas para generar la clasificación
model.add(Dense(2))
model.add(Activation('softmax'))

# compile model and initialize weights
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# let's summarize the CNN architectures along with the number of model weights
model.summary()

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

In [ ]:
# train the model
history=model.fit(X_train, Y_train,
                  validation_data=(X_val,Y_val),
                  batch_size=64, 
                  epochs=200,
                  verbose=0,
                  shuffle=True) #shuffle: Boolean (whether to shuffle the training data before each epoch)

In [ ]:
# plot the development of the accuracy and loss during training
plt.figure(figsize=(12,4))
plt.subplot(1,2,(1))
plt.plot(history.history['accuracy'],linestyle='-.')
plt.plot(history.history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.subplot(1,2,(2))
plt.plot(history.history['loss'],linestyle='-.')
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right')


In [ ]:
# get the leared weights and display them as image
conv_filter=model.get_weights()[0]
conv_filter.shape
conv_filter=np.squeeze(conv_filter, axis=2)

plt.figure()
plt.imshow(conv_filter[:,:,0],"gray")
plt.axis("off")
plt.show()

In [ ]:
len(model.get_weights())

In [ ]:
all_weights = model.get_weights()

In [ ]:
print(all_weights[0].shape)
print(all_weights[1].shape)
print(all_weights[2].shape)
print(all_weights[3].shape)

In [ ]:
# conv_filter=model.get_weights()[0]
print(conv_filter.shape)

In [ ]:
ddprint(conv_filter*255)